In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from os import sys

In [ ]:



# Function to calculate gravitational force between fragments
def gravitational_force(i, j, positions, velocities, masses):
    r_ij = np.array([positions[i, 0] - positions[j, 0],
                     positions[i, 1] - positions[j, 1],
                     positions[i, 2] - positions[j, 2]])
    distance = np.linalg.norm(r_ij)
    if distance == 0:
        return np.array([0.0, 0.0, 0.0])
    force_magnitude = G * masses[i] * masses[j] / distance**2
    force_vector = force_magnitude * r_ij / distance
    return force_vector

In [ ]:
# Function to apply collision response
def handle_collisions(i, j, positions, velocities, masses):
    r_ij = np.array([positions[i, 0] - positions[j, 0],
                     positions[i, 1] - positions[j, 1],
                     positions[i, 2] - positions[j, 2]])
    distance = np.linalg.norm(r_ij)
    if distance < 1e3:  # Assume fragments are colliding if distance is small enough
        # Use elastic collision formulas to update velocities
        v_rel = velocities[i] - velocities[j]
        normal = r_ij / distance
        # v_dot_n = np.dot(v_rel, normal)
        v_dot_n = np.dot(v_rel, normal)[0]
        # print(f"{type(v_dot_n)}")
        if v_dot_n < 0:
            # Update velocities based on conservation of momentum
            m1, m2 = masses[i], masses[j]
            v1_new = velocities[i] - (2 * m2 / (m1 + m2)) * v_dot_n * normal
            v2_new = velocities[j] - (2 * m1 / (m1 + m2)) * v_dot_n * normal
            velocities[i] = v1_new
            velocities[j] = v2_new

In [ ]:
# Function to apply mass decay
def apply_mass_decay(masses, decay_rate=0.001):
    return masses * (1 - decay_rate)

In [ ]:
a=np.random.uniform(0,20,5)
print(a)

[18.88157216 13.76158178 13.4570768  18.52066956  0.29557892]


In [ ]:
# Constants
# num_fragments = 1000        # Number of fragments created after explosion
num_fragments = 100        # Number of fragments created after explosion
total_mass = 1e30            # Total mass (in kg)
min_mass = 1e23              # Minimum mass of individual fragments (in kg)
max_mass = 1e26              # Maximum mass of individual fragments (in kg)
max_velocity = 1e4           # Maximum velocity of fragments (in m/s)
explosion_radius = 1e6       # Explosion radius (in meters)
G = 6.67430e-11              # Gravitational constant (m^3 kg^-1 s^-2)
k_B = 1.380649e-23           # Boltzmann constant (J/K)
T = 1e5                       # Temperature (Kelvin) for Maxwell-Boltzmann distribution
mass_decay_rate=0.001
conic_section_split=5

# Generate random fragment masses (uniform distribution)
mass_distribution=np.random.uniform(min_mass,max_mass,num_fragments)
# mass_distribution=np.random.random(min_mass,max_mass,num_fragments)
mass_distribution/=sum(mass_distribution)
fragment_masses=np.array([mass_distribution[i]*total_mass for i in range(len(mass_distribution))])
# fragment_masses = np.random.uniform(min_mass, max_mass, num_fragments)

# if sum(fragment_masses)>(total_mass+1):
#     sys.exit("Wrong Model")
# else:
#   print(f"total_mass:{total_mass},sum(fragment_masses):{sum(fragment_masses)}")
#   print(f"fragment_masses:{fragment_masses}")

# Generate velocities using the Maxwell-Boltzmann distribution
theta = np.random.uniform(0, 2 * np.pi, num_fragments)  # Random azimuthal angle
phi = np.random.uniform(0, np.pi, num_fragments)         # Random polar angle
# theta_list=[np.random.uniform(0, 2 * np.pi, num_fragments)]
speed = np.random.normal(0, np.sqrt(k_B * T / fragment_masses), num_fragments)  # Maxwell-Boltzmann distribution
theta_cutoff=[(2*np.pi/conic_section_split)*i for i in range(conic_section_split+1)]
print(f"theta_cutoff:{theta_cutoff}")
phi_cutoff=[(np.pi/conic_section_split)*i for i in range(conic_section_split+1)]
print(f"phi_cutoff:{phi_cutoff}")
theta_list,phi_list=[],[]
for i in range(len(theta_cutoff)-1):
  # temp_theta_list=[angle for angle in theta if angle>theta_cutoff[i] and angle<theta_cutoff[i+1]]
  temp_theta_list=list(np.random.uniform(theta_cutoff[i],theta_cutoff[i+1],int(num_fragments/conic_section_split)))
  temp_phi_list=list(np.random.uniform(phi_cutoff[i],phi_cutoff[i+1],int(num_fragments/conic_section_split)))
  # temp_phi_list=[angle for angle in phi if angle>phi_cutoff[i] and angle<phi_cutoff[i+1]]
  theta_list.append(temp_theta_list)
  phi_list.append(temp_phi_list)
length_list=[len(theta_list[i]) for i in range(len(theta_list))]
length_list=np.cumsum(length_list)
print(f"old length_list:{length_list}")
length_list=[0]+list(length_list)
print(f"new length_list:{length_list}")
# speed_list=[speed[len()]]
print(f"theta_list:{theta_list}")
print(f"len(theta_list):{len(theta_list)})")
print(f"len(phi_list):{len(phi_list)}")

# Calculate velocity components in 3D space (spherical to Cartesian conversion)
vx = speed * np.sin(phi) * np.cos(theta)  # X velocity component
vy = speed * np.sin(phi) * np.sin(theta)  # Y velocity component
vz = speed * np.cos(phi)                 # Z velocity component

# print(f"len(length_list):{len(length_list)}")
# for i in range(len(theta_list)):
#   # print(f"i:{i}")
#   # print(f"{length_list[i+1]},{len(phi_list[i])},{len(theta_list[i])}")
#   print(f"{length_list[i+1]-length_list[i]},{len(phi_list[i])},{len(theta_list[i])}")
#   # print(f"{length_list[i]},{len(phi_list[i])},{len(theta_list[i])}")

vx_initial_list=[speed[length_list[i]:length_list[i+1]]*np.sin(phi_list[i])*np.cos(theta_list[i]) for i in range(len(theta_list))]
vy_initial_list=[speed[length_list[i]:length_list[i+1]]*np.sin(phi_list[i])*np.sin(theta_list[i]) for i in range(len(theta_list))]
vz_initial_list=[speed[length_list[i]:length_list[i+1]]*np.cos(phi_list[i]) for i in range(len(theta_list))]
print(f"len(vx_list):{len(vx_initial_list)}")
print(f"len(vy_list):{len(vy_initial_list)}")
print(f"len(vz_list):{len(vz_initial_list)}")
# Generate random initial positions (within the explosion radius)
r = np.random.uniform(0, explosion_radius, num_fragments)
x_initial = r * np.sin(phi) * np.cos(theta)
y_initial = r * np.sin(phi) * np.sin(theta)
z_initial = r * np.cos(phi)

x_initial_list=[r[length_list[i]:length_list[i+1]]*np.sin(phi_list[i])*np.cos(theta_list[i]) for i in range(len(theta_list))]
y_initial_list=[r[length_list[i]:length_list[i+1]]*np.sin(phi_list[i])*np.sin(theta_list[i]) for i in range(len(theta_list))]
z_initial_list=[r[length_list[i]:length_list[i+1]]*np.cos(phi_list[i]) for i in range(len(theta_list))]
initial_mass_list=[fragment_masses[length_list[i]:length_list[i+1]] for i in range(len(theta_list))]
print(f"len(x_initial_list):{len(x_initial_list)}")

# Time simulation parameters
time_steps = 1000   # Number of simulation time steps
dt = 1000           # Time step in seconds

mass_out_eva_list=[]
for i in range(conic_section_split):
  mass_list=[]
  for j in range(len(x_initial_list[i])):
    initial_position=np.linalg.norm([x_initial_list[i][j],y_initial_list[i][j],z_initial_list[i][j]])
    initial_speed=np.linalg.norm([vx_initial_list[i][j],vy_initial_list[i][j],vz_initial_list[i][j]])
    time_inside_eva=int((explosion_radius-initial_position)/dt)
    for k in range(time_inside_eva):
      initial_mass_list[i][j]=apply_mass_decay(initial_mass_list[i][j],mass_decay_rate)
    mass_list.append(initial_mass_list[i][j])
  mass_out_eva_list.append(mass_list)

print(f"len(mass_out_eva_list):{len(mass_out_eva_list)}")
print(f"mass_out_eva_list:{mass_out_eva_list}")

store_df=pd.DataFrame({'x':x_initial_list,'y':y_initial_list,'z':z_initial_list,'vx':vx_initial_list,'vy':vy_initial_list,'vz':vz_initial_list,'mass':mass_out_eva_list})
store_df.to_csv('initial_fragments.csv',index=False)

'''
sys.exit()
fragments_config=pd.DataFrame({'mass':fragment_masses,'x':x_initial,'y':y_initial,'z':z_initial,'vx':vx,'vy':vy,'vz':vz})

for i in range(num_fragments):
  initial_position=np.linalg.norm([x_initial[i],y_initial[i],z_initial[i]])
  initial_speed=np.linalg.norm([vx[i],vy[i],vz[i]])
  time_inside_eva=int((explosion_radius-initial_position)/dt)
  for j in range(time_inside_eva):
    fragment_masses[i]=apply_mass_decay(fragment_masses[i],mass_decay_rate)


#### Till here only

print(f"Initial {x_initial.shape,y_initial.shape,z_initial.shape,vx.shape,vy.shape,vz.shape}")
fragments_config=pd.DataFrame({'mass':fragment_masses,'x':x_initial,'y':y_initial,'z':z_initial,'vx':vx,'vy':vy,'vz':vz})
fragments_config.to_csv('fragments_config.csv',index=False)
sys.exit()

# Initialize arrays to store the positions and velocities of the fragments
x_positions = np.zeros((time_steps, num_fragments))
y_positions = np.zeros((time_steps, num_fragments))
z_positions = np.zeros((time_steps, num_fragments))
vx_all = np.zeros((time_steps, num_fragments))
vy_all = np.zeros((time_steps, num_fragments))
vz_all = np.zeros((time_steps, num_fragments))

# Set initial positions and velocities
x_positions[0, :] = x_initial
y_positions[0, :] = y_initial
z_positions[0, :] = z_initial
vx_all[0, :] = vx
vy_all[0, :] = vy
vz_all[0, :] = vz


positions=np.zeros((num_fragments,3))
# Simulate the motion of the fragments over time
for t in range(1, time_steps):
    # Update positions based on velocities
    x_positions[t, :] = x_positions[t-1, :] + vx_all[t-1, :] * dt
    y_positions[t, :] = y_positions[t-1, :] + vy_all[t-1, :] * dt
    z_positions[t, :] = z_positions[t-1, :] + vz_all[t-1, :] * dt

    # Apply gravitational forces between all pairs of fragments
    for i in range(num_fragments):
        total_force = np.zeros(3)
        for j in range(num_fragments):
            if i != j:
                # print(f"{gravitational_force.shape},{x_positions.shape},{fragment_masses.shape}")
                # print(f"{x_positions.shape},{vx_all.shape},{fragment_masses.shape}")
                # print(f"{x_positions[:,1].shape}")
                # positions=np.array([x_positions[:,0:1],y_positions[:,0:1],z_positions[:,0:1]])
                # positions=np.array([x_positions[:,1],y_positions[:,1],z_positions[:,1]])
                positions[:,0]=x_positions[t-1,:];positions[:,1]=y_positions[t-1,:];positions[:,2]=z_positions[t-1,:];
                # print(f"{positions.shape}")
                # total_force += gravitational_force(i, j, x_positions[t-1, :], vx_all[t-1, :], fragment_masses)
                total_force += gravitational_force(i, j, positions, vx_all[t-1, :], fragment_masses)

        # Update velocities based on forces (F = ma, so a = F/m)
        vx_all[t, i] = vx_all[t-1, i] + total_force[0] * dt / fragment_masses[i]
        vy_all[t, i] = vy_all[t-1, i] + total_force[1] * dt / fragment_masses[i]
        vz_all[t, i] = vz_all[t-1, i] + total_force[2] * dt / fragment_masses[i]

    # Apply collisions between fragments
    for i in range(num_fragments):
        for j in range(i + 1, num_fragments):  # Only check each pair once
            # handle_collisions(i, j, x_positions[t, :], vx_all[t, :], fragment_masses)
            positions[:,0]=x_positions[t-1,:];positions[:,1]=y_positions[t-1,:];positions[:,2]=z_positions[t-1,:];
            handle_collisions(i, j, positions, vx_all[t, :], fragment_masses)

    # Apply mass decay
    fragment_masses = apply_mass_decay(fragment_masses)

# Visualize the final positions of the fragments in 3D space
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot of the final positions of the fragments
ax.scatter(x_positions[-1, :], y_positions[-1, :], z_positions[-1, :], c=fragment_masses, cmap='viridis', s=1, alpha=0.7)

# Labels and title
ax.set_xlabel('X Position (m)')
ax.set_ylabel('Y Position (m)')
ax.set_zlabel('Z Position (m)')
ax.set_title('Mass Fragments Distribution After Explosion with All Effects')

plt.show()
'''

theta_cutoff:[0.0, 1.2566370614359172, 2.5132741228718345, 3.7699111843077517, 5.026548245743669, 6.283185307179586]
phi_cutoff:[0.0, 0.6283185307179586, 1.2566370614359172, 1.8849555921538759, 2.5132741228718345, 3.141592653589793]
old length_list:[ 20  40  60  80 100]
new length_list:[0, 20, 40, 60, 80, 100]
theta_list:[[0.20069436226299842, 0.26358429165561503, 0.401676825397618, 1.1097110280826086, 0.04101246970141294, 0.7679325594632053, 1.0855360830892307, 0.1259242650116036, 1.065257738538247, 0.2758503408462669, 0.4125774389283316, 1.1439119565172646, 0.7724323945837238, 1.0397932099788538, 0.865744279797245, 0.2108631557939647, 0.690707823220943, 0.5932722056497688, 0.07852909775526118, 0.16658208920559534], [1.8067720418221556, 2.202521085998864, 2.2581300641690425, 1.7181416648439423, 2.496658055891191, 2.429839836896355, 1.575539891222421, 1.8794691580727692, 1.6454789297462995, 1.323625098481159, 1.5324982683656874, 2.0598359999748683, 2.2875511357215093, 1.56498010055759,

'\nsys.exit()\nfragments_config=pd.DataFrame({\'mass\':fragment_masses,\'x\':x_initial,\'y\':y_initial,\'z\':z_initial,\'vx\':vx,\'vy\':vy,\'vz\':vz})\n\nfor i in range(num_fragments):\n  initial_position=np.linalg.norm([x_initial[i],y_initial[i],z_initial[i]])\n  initial_speed=np.linalg.norm([vx[i],vy[i],vz[i]])\n  time_inside_eva=int((explosion_radius-initial_position)/dt)\n  for j in range(time_inside_eva):\n    fragment_masses[i]=apply_mass_decay(fragment_masses[i],mass_decay_rate)\n\n\n#### Till here only\n\nprint(f"Initial {x_initial.shape,y_initial.shape,z_initial.shape,vx.shape,vy.shape,vz.shape}")\nfragments_config=pd.DataFrame({\'mass\':fragment_masses,\'x\':x_initial,\'y\':y_initial,\'z\':z_initial,\'vx\':vx,\'vy\':vy,\'vz\':vz})\nfragments_config.to_csv(\'fragments_config.csv\',index=False)\nsys.exit()\n\n# Initialize arrays to store the positions and velocities of the fragments\nx_positions = np.zeros((time_steps, num_fragments))\ny_positions = np.zeros((time_steps, num_fr

In [ ]:
a=[4,3,1]
b=np.linalg.norm(a)
print(b)

5.0990195135927845


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Constants
num_fragments = 1000        # Number of fragments created after explosion
total_mass = 1e30            # Total mass (in kg)
min_mass = 1e23              # Minimum mass of individual fragments (in kg)
max_mass = 1e26              # Maximum mass of individual fragments (in kg)
max_velocity = 1e4           # Maximum velocity of fragments (in m/s)
explosion_radius = 1e6       # Explosion radius (in meters)
G = 6.67430e-11              # Gravitational constant (m^3 kg^-1 s^-2)
k_B = 1.380649e-23           # Boltzmann constant (J/K)
T = 1e5                       # Temperature (Kelvin) for Maxwell-Boltzmann distribution

# Generate random fragment masses (uniform distribution)
fragment_masses = np.random.uniform(min_mass, max_mass, num_fragments)

# Generate velocities using the Maxwell-Boltzmann distribution
theta = np.random.uniform(0, 2 * np.pi, num_fragments)  # Random azimuthal angle
phi = np.random.uniform(0, np.pi, num_fragments)         # Random polar angle
speed = np.random.normal(0, np.sqrt(k_B * T / fragment_masses), num_fragments)  # Maxwell-Boltzmann distribution

# Calculate velocity components in 3D space (spherical to Cartesian conversion)
vx = speed * np.sin(phi) * np.cos(theta)  # X velocity component
vy = speed * np.sin(phi) * np.sin(theta)  # Y velocity component
vz = speed * np.cos(phi)                 # Z velocity component

# Generate random initial positions (within the explosion radius)
r = np.random.uniform(0, explosion_radius, num_fragments)
x_initial = r * np.sin(phi) * np.cos(theta)
y_initial = r * np.sin(phi) * np.sin(theta)
z_initial = r * np.cos(phi)

# Time simulation parameters
time_steps = 1000   # Number of simulation time steps
dt = 1000           # Time step in seconds

# Initialize arrays to store the positions and velocities of the fragments
x_positions = np.zeros((time_steps, num_fragments))
y_positions = np.zeros((time_steps, num_fragments))
z_positions = np.zeros((time_steps, num_fragments))
vx_all = np.zeros((time_steps, num_fragments))
vy_all = np.zeros((time_steps, num_fragments))
vz_all = np.zeros((time_steps, num_fragments))

# Set initial positions and velocities
x_positions[0, :] = x_initial
y_positions[0, :] = y_initial
z_positions[0, :] = z_initial
vx_all[0, :] = vx
vy_all[0, :] = vy
vz_all[0, :] = vz

# Function to calculate gravitational force between fragments
def gravitational_force(i, j, positions, velocities, masses):
    print(positions)
    r_ij = np.array([positions[i, 0] - positions[j, 0],
                     positions[i, 1] - positions[j, 1],
                     positions[i, 2] - positions[j, 2]])
    distance = np.linalg.norm(r_ij)
    if distance == 0:
        return np.array([0.0, 0.0, 0.0])
    force_magnitude = G * masses[i] * masses[j] / distance**2
    force_vector = force_magnitude * r_ij / distance
    return force_vector

# Function to apply collision response
def handle_collisions(i, j, positions, velocities, masses):
    r_ij = np.array([positions[i, 0] - positions[j, 0],
                     positions[i, 1] - positions[j, 1],
                     positions[i, 2] - positions[j, 2]])
    distance = np.linalg.norm(r_ij)
    if distance < 1e3:  # Assume fragments are colliding if distance is small enough
        # Use elastic collision formulas to update velocities
        v_rel = velocities[i] - velocities[j]
        normal = r_ij / distance
        v_dot_n = np.dot(v_rel, normal)
        if v_dot_n < 0:
            # Update velocities based on conservation of momentum
            m1, m2 = masses[i], masses[j]
            v1_new = velocities[i] - (2 * m2 / (m1 + m2)) * v_dot_n * normal
            v2_new = velocities[j] - (2 * m1 / (m1 + m2)) * v_dot_n * normal
            velocities[i] = v1_new
            velocities[j] = v2_new

# Function to apply mass decay
def apply_mass_decay(masses, decay_rate=0.001):
    return masses * (1 - decay_rate)

# Simulate the motion of the fragments over time
for t in range(1, time_steps):
    # Update positions based on velocities
    x_positions[t, :] = x_positions[t-1, :] + vx_all[t-1, :] * dt
    y_positions[t, :] = y_positions[t-1, :] + vy_all[t-1, :] * dt
    z_positions[t, :] = z_positions[t-1, :] + vz_all[t-1, :] * dt

    # Apply gravitational forces between all pairs of fragments
    for i in range(num_fragments):
        total_force = np.zeros(3)
        for j in range(num_fragments):
            if i != j:
                total_force += gravitational_force(i, j, x_positions[t-1, :], vx_all[t-1, :], fragment_masses)

        # Update velocities based on forces (F = ma, so a = F/m)
        vx_all[t, i] = vx_all[t-1, i] + total_force[0] * dt / fragment_masses[i]
        vy_all[t, i] = vy_all[t-1, i] + total_force[1] * dt / fragment_masses[i]
        vz_all[t, i] = vz_all[t-1, i] + total_force[2] * dt / fragment_masses[i]

    # Apply collisions between fragments
    for i in range(num_fragments):
        for j in range(i + 1, num_fragments):  # Only check each pair once
            handle_collisions(i, j, x_positions[t, :], vx_all[t, :], fragment_masses)

    # Apply mass decay
    fragment_masses = apply_mass_decay(fragment_masses)

# Visualize the final positions of the fragments in 3D space
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot of the final positions of the fragments
ax.scatter(x_positions[-1, :], y_positions[-1, :], z_positions[-1, :], c=fragment_masses, cmap='viridis', s=1, alpha=0.7)

# Labels and title
ax.set_xlabel('X Position (m)')
ax.set_ylabel('Y Position (m)')
ax.set_zlabel('Z Position (m)')
ax.set_title('Mass Fragments Distribution After Explosion with All Effects')

plt.show()


[-4.02242100e+05 -1.34948500e+05 -3.57339362e+05 -1.40532298e+05
 -3.62029164e+05  1.55166268e+04 -1.23515754e+05  2.89285667e+04
 -3.99800395e+05 -4.34845564e+05 -3.20814852e+04 -3.85390132e+04
 -9.01527606e+05 -6.54389907e+05 -9.15156051e+05 -9.44328545e+05
  3.04031205e+03 -2.22854223e+05  2.26423391e+04  3.90788039e+05
  4.68630357e+05  5.83684924e+04  2.42394043e+05 -1.83230025e+05
 -2.15780476e+05 -7.10244619e+04  5.67728004e+05 -3.80090014e+05
 -1.76859264e+05 -1.87339565e+05  2.28252381e+05  5.71684128e+04
  8.10586759e+05 -4.47978138e+05 -6.17361872e+04  3.76293245e+05
 -5.23210446e+05  3.33571825e+05  2.37868071e+04  3.97761336e+05
 -3.38397030e+03 -1.13619158e+04 -3.56494127e+05  3.51552099e+05
  2.46619098e+05  2.12607905e+04  1.00385930e+05  1.92968994e+04
  2.03217003e+05  1.25622221e+04 -3.11198908e+05 -1.71752620e+05
  5.54260956e+04  1.06848575e+05  3.52309218e+05  4.01463114e+05
 -3.94737136e+05 -8.67353511e+04  4.98060121e+05 -6.06444251e+04
  9.96440160e+04 -4.06143

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed